In [ ]:
import pytorch_lightning as pl
import torch as pt

# MUST import tensorflow SECOND or else you will encounter ambiguous error!
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import layers as L

In [ ]:
model = keras.models.load_model('PCDNNV2_decomp/regressor')

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
    show_trainable=False,
)

In [ ]:
import os
from glob import glob
import random

print('PCDNNV2_decomp:', os.listdir('./PCDNNV2_decomp'))

Inv_ckpt = random.choice(glob('../CT_logs_Mu/Inv*/version_*/checkpoints/*.ckpt'))
CPV_source_ckpt = random.choice(glob('../CT_logs_Mu/Real*/version_*/checkpoints/*.ckpt'))
Souener_ckpt = random.choice(glob('../CT_logs_Mu/Souener*/version_*/checkpoints/*.ckpt'))
print('Inv checkpoint: ', Inv_ckpt)
print('CPV_source checkpoint: ', CPV_source_ckpt)
print('Energy_source checkpoints', Souener_ckpt)

In [ ]:
import ONNX_export
CPV_source_model = ONNX_export.export_CT_model_for_ablate(CPV_source_ckpt)
Souener_model = ONNX_export.export_CT_model_for_ablate(Souener_ckpt)
Inv_model = ONNX_export.export_CT_model_for_ablate(Inv_ckpt, add_hard_l1_constraint=True)

In [ ]:
def make_aggregate_regressor(CPV_source_ckpt: str, souener_ckpt: str, inv_ckpt: str):
    """ makes aggergate regressor with the same interfaces as CT V1 from individual models given from CT V2"""
    print('Inv checkpoint: ', Inv_ckpt)
    print('CPV_source checkpoint: ', CPV_source_ckpt)
    print('Energy_source checkpoints', Souener_ckpt)
    
    import ONNX_export
    CPV_source_model = ONNX_export.export_CT_model_for_ablate(CPV_source_ckpt)
    Souener_model = ONNX_export.export_CT_model_for_ablate(souener_ckpt)
    Inv_model = ONNX_export.export_CT_model_for_ablate(inv_ckpt, add_hard_l1_constraint=True)
    # hard_l1_constraint is needed for Inv model only due to constraints on Yis
    
    assert Souener_model.output_shape[1]==1, 'invalid souener model!'
    assert CPV_source_model.input_shape==CPV_source_model.output_shape, 'invalid CPV_Source model!'
    assert Inv_model.input_shape[1]<Inv_model.output_shape[1], 'invalid inverse model!'
    assert CPV_source_model.input_shape==Souener_model.input_shape==Inv_model.input_shape, 'incompatible models!'

    input_shape = CPV_source_model.input_shape[1:]
    print('input_shape: ', input_shape)
    input_=L.Input(shape=CPV_source_model.input_shape[1:], name='input_1')
    
    dynamic_source_prediction = L.Rescaling(1.0, name='dynamic_source_prediction')(CPV_source_model(input_))
    static_source_prediction = L.Concatenate(name='static_source_prediction')([Souener_model(input_), Inv_model(input_)])
    
    outputs={'static_source_prediction': static_source_prediction, 'dynamic_source_prediction': dynamic_source_prediction}
    aggergate_regressor = keras.models.Model(inputs=input_, outputs=outputs, name='V2_aggergate_regressor')
    print('aggregate regressor summary: ')
    aggergate_regressor.summary()
    print('aggregate regressor input_shape: ', aggergate_regressor.input_shape)
    print('aggregate regressor output_shape: ', aggergate_regressor.output_shape)
    tf.keras.utils.plot_model(aggergate_regressor, to_file='aggregate_regressor.png', show_shapes=True, show_dtype=True)
    return aggergate_regressor
aggergate_regressor = make_aggregate_regressor(CPV_source_ckpt, Souener_ckpt, Inv_ckpt)

In [ ]:
tf.keras.utils.plot_model(
    aggergate_regressor,
    to_file='aggregate_regressor.png',
    show_shapes=True,
    show_dtype=True
)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

dummy_inputs = np.random.randn(100,25)
plt.imshow(dummy_inputs)
plt.title('dummy_inputs')
plt.show()

def inspect_outputs(outputs, model_name: str):
    plt.imshow(outputs['dynamic_source_prediction'])
    plt.title(f'CPV_source-{model_name}_outputs')
    plt.show()
    plt.imshow(outputs['static_source_prediction'][:,1:])
    plt.title(f'Inv-{model_name}_outputs')
    plt.show()
    plt.hist(np.asarray(outputs['static_source_prediction'][:,:1]).squeeze())
    plt.title(f'Souener-{model_name}_outputs')
    plt.show()
inspect_outputs(model(dummy_inputs[:, :13]), 'org')

In [ ]:
agg_outputs = aggergate_regressor(dummy_inputs)
inspect_outputs(agg_outputs, 'agg')

In [ ]:
# manually combine invidual small model preds into agg prediction to compare!
CPV_source_pred = CPV_source_model(dummy_inputs)
souener_pred = Souener_model(dummy_inputs).numpy().reshape(-1,1)
inv_pred = Inv_model(dummy_inputs)
manually_constructed_outputs = {'static_source_prediction': np.concatenate([souener_pred, inv_pred], axis=1), 'dynamic_source_prediction': CPV_source_pred}

inspect_outputs(manually_constructed_outputs, 'manual')

In [ ]:
agg_outputs['static_source_prediction'][:, 1:]